In [ ]:
import numpy as np
from qiskit import QuantumCircuit, Aer, execute
from qiskit.quantum_info import Statevector, Operator, Pauli
import random
import torch

config = {
    "qubits": 4,
    "layers": 3,
}

def circuit(params):
    qc = QuantumCircuit(config['qubits'])
    
    z1 = random.uniform(-1, 1)
    z2 = random.uniform(-1, 1)
    for i in range(config["qubits"]):
        qc.ry(np.arcsin(z1), i)
        qc.rz(np.arcsin(z2), i)
    
    for l in range(config['layers']):
        for i in range(config['qubits']):
            qc.ry(params[i], i)
        for i in range(config['qubits'] - 1):
            qc.cx(i, i + 1)
            qc.rz(params[i + config['qubits']], i + 1)
            qc.cx(i, i + 1)
    
    return qc

def expectation_values(circuit):
    backend = Aer.get_backend('statevector_simulator')
    job = execute(circuit, backend, shots = 5000)
    state = job.result().get_statevector()
    
    expectations = []
    for i in range(config['qubits']):
        pauli_list = ['I'] * config['qubits']
        pauli_list[i] = 'Z'
        z_pauli = Pauli(''.join(pauli_list))

        expectation = Statevector(state).expectation_value(z_pauli).real
        expectations.append(expectation)
    
    return expectations[0]

params = np.random.rand(config['qubits'] * 2) 
qc = circuit(params)
expectations = expectation_values(qc)

print(expectations)

In [ ]:
x = torch.randn(1, 3, 64, 64)

Ɛ = torch.zeros_like(x)

for i in range(x.shape[1]): 
    for j in range(x.shape[2]):  
        for k in range(x.shape[3]):  
            values = expectation_values(qc)
            Ɛ[:, i, j, k] = values

print(Ɛ)
print(Ɛ.shape)

In [ ]:
x = torch.randn(1, 3, 64, 64)
Ɛ = torch.zeros_like(x)

for i in range(x.shape[1]): 
    for j in range(x.shape[2]):
        values = []
        for k in range(0, x.shape[3]):
            if k % config["qubits"] == 0:  
                values = expectation_values(qc)
                random.shuffle(values)
            Ɛ[:, i, j, k] = values[k % config["qubits"]] 

print(Ɛ)
print(Ɛ.shape)